In [1]:
from IPython.display import display
import math
import piplite
await piplite.install("bqplot")
await piplite.install("ipywidgets")
import bqplot
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root, fsolve

In [2]:
n = 0.9
k1 = [0.604, 3.765]
k1b = 0.05
Km = 40
s = 0
m = [5,0]
p = [30,0,200]
tau_mRNA = 2
tau_prot = 10
kd_m = [math.log(2)/tau_mRNA, math.log(2)/3]
kd_p = [math.log(2)/tau_prot, math.log(2)/11]
k_tl = [0.34, 2.017]
k_f = 0.5  
k_d = 0.025

In [3]:
def dCas_NOT_model(t, data, k1, k1b, Km, kd_m, kd_p, k_f, k_d, k_tl, s, n):
    m = data[:2]
    p = data[2:]
    dm0 = -kd_m[0]*m[0] + k1[0]
    dm1 = -kd_m[1]*m[1] + k1[1]*(Km**n)/((Km**n)+(p[1])**n) +  k1b
    dp0 = -kd_p[0]*p[0] + k_tl[0]*m[0] + k_d*p[1] - k_f*p[0]*s
    dp1 = -k_d*p[1] + k_f*p[0]*s - kd_p[0]*p[1]
    dp2 = -kd_p[1]*p[2] + k_tl[1]*m[1]
    return [dm0,dm1,dp0,dp1,dp2]

In [4]:
t0 = 0.0
tf = 200.0
time = np.linspace(0.0,200.0,2000)
initval = np.array([*m,*p])
res = solve_ivp(dCas_NOT_model, (t0,tf), initval, method='RK45', t_eval=time, args=(k1,k1b,Km,kd_m,kd_p,k_f,k_d,k_tl,s,n))
res = dict(res)
time = res['t'] 
y = res['y']
yGFP =y[4]


In [5]:
fig = plt.figure(title="dCas9 NOT-gate model", legend_location="top-left")
fig.layout.height = '500px'
fig.layout.width = '650px'

line_chart = plt.plot(x=time, y=yGFP, labels=["GFP"], colors=['green'],
                     display_legend=True)

plt.xlabel("Time, min")
plt.ylabel("Number of molecules")

In [6]:
def update_plot(tM1, tM2, tP1, tP2, CmCas, CmGFP, Cs, CpCas_f, CpGFP, n):
    s = Cs
    kd_m = [math.log(2)/tM1, math.log(2)/tM2]
    kd_p = [math.log(2)/tP1, math.log(2)/tP2]
    t0 = 0.0
    tf = 200.0
    time = np.linspace(0.0,200.0,2000)
    initval = np.array([CmCas, CmGFP, CpCas_f, p[1], CpGFP])
    res = solve_ivp(dCas_NOT_model, (t0,tf), initval, method='RK45', t_eval=time, args=(k1,k1b,Km,kd_m,kd_p,k_f,k_d,k_tl,s,n))
    res = dict(res)
    time = res['t']
    y = res['y']
    yGFP = y[4]
    line_chart.y = yGFP
    

In [7]:
#widgets
wdj = widgets.interactive(update_plot, tM1 = widgets.FloatText(value=tau_mRNA, description="tM_Cas:", disabled=False),
tM2 = widgets.FloatText(value=3, description="tM_GFP:", disabled=False),                           
tP1 = widgets.FloatText(value=tau_prot, description="tP_Cas:", disabled=False),
tP2 = widgets.FloatText(value=11, description="tP_GFP:", disabled=False),
CmCas = widgets.FloatText(value=m[0], description="CmCas:", disabled=False),
CmGFP = widgets.FloatText(value=m[1], description="CmGFP:", disabled=False),
Cs = widgets.FloatText(value=s, description="C_sgRNA:", disabled=False),
CpCas_f = widgets.FloatText(value=p[0], description="CpCas_free:", disabled=False),
CpGFP = widgets.FloatText(value=p[2], description="CpGFP:", disabled=False),
n = widgets.FloatSlider(value=n, min=0.9, max=2.0, step=0.1, description="n:", disabled=False))

In [8]:
display(widgets.HBox([fig,wdj]))

<h2>Finding the fixed points</h2>

In [9]:
#Finding fixed points
def eq_points_equations(X, k1, k1b, Km, kd_m, kd_p, k_f, k_d, k_tl, s, n):
    Xm1, Xm2, Xp1, Xp2, Xp3 = X
    f = [
        Xm1 - k1[0]/kd_m[0],
        Xm2 - k1[1]*(Km**n)/(((Km**n)+(p[1])**n)*kd_m[1]) -  k1b/kd_m[1],
        Xp1 - k_tl[0]*m[0]/kd_p[0] - k_d*p[1]/kd_p[0] + k_f*p[0]*s/kd_p[0],
        Xp2 - k_f*p[0]*s/(k_d + kd_p[0]),
        Xp3 - k_tl[1]*m[1]/kd_p[1]
    
    ]
    return f

points = root(eq_points_equations, initval, args=(k1, k1b, Km, kd_m, kd_p, k_f, k_d, k_tl, s, n))
points

    fjac: array([[-1.00000000e+00,  2.90956495e-15, -9.64723093e-16,
         0.00000000e+00, -3.52426421e-14],
       [-2.90952076e-15, -1.00000000e+00,  4.89061916e-15,
         0.00000000e+00,  1.78662640e-13],
       [ 9.64673151e-16, -4.89064877e-15, -1.00000000e+00,
         0.00000000e+00, -5.92303984e-14],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.00000000e+00,  0.00000000e+00],
       [ 3.52414246e-14, -1.78665105e-13,  5.92303984e-14,
         0.00000000e+00, -1.00000000e+00]])
     fun: array([0.00000000e+00, 2.77555756e-17, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])
 message: 'The solution converged.'
    nfev: 8
     qtf: array([ 7.10453918e-12, -3.60138863e-11,  1.19406707e-11,  0.00000000e+00,
        4.36216396e-10])
       r: array([-1.00000000e+00,  5.81978042e-15, -1.92950040e-15,  0.00000000e+00,
       -7.04887537e-14, -1.00000000e+00,  9.78123832e-15,  0.00000000e+00,
        3.57311403e-13, -1.00000000e+00,  0.00000000e+00

This system has one fixed point/stationary state

In [10]:
fig1 = plt.figure(title="Phase portrait 1", legend_location="top-left")
phase1 = plt.plot(y[3], y[4])
plt.ylabel("GFP, number of proteins")
plt.xlabel("dCas9:sgRNA, number of complexes")
fig1.layout.height = '500px'
fig1.layout.width = '500px'

In [11]:
def update_phase1(tM1, tM2, tP1, tP2, Cs, Km, n):
    s = Cs
    kd_m = [math.log(2)/tM1, math.log(2)/tM2]
    kd_p = [math.log(2)/tP1, math.log(2)/tP2]
    t0 = 0.0
    tf = 200.0
    time = np.linspace(0.0,200.0,2000)
    initval = np.array([*m,*p])
    res = solve_ivp(dCas_NOT_model, (t0,tf), initval, method='RK45', t_eval=time, args=(k1,k1b,Km,kd_m,kd_p,k_f,k_d,k_tl,s,n))
    res = dict(res)
    time = res['t']
    y = res['y']
    yGFP = y[4]
    ydCas_s = y[3]
    phase1.y = yGFP
    phase1.x = ydCas_s

In [12]:
wdj1 = widgets.interactive(update_phase1, tM1 = widgets.FloatText(value=tau_mRNA, description="tM_Cas:", disabled=False),
tM2 = widgets.FloatText(value=3, description="tM_GFP:", disabled=False),                           
tP1 = widgets.FloatText(value=tau_prot, description="tP_Cas:", disabled=False),
tP2 = widgets.FloatText(value=11, description="tP_GFP:", disabled=False),
Cs = widgets.FloatText(value=s, description="C_sgRNA:", disabled=False),                           
Km = widgets.FloatText(value=40, description="Km:", disabled=False),                          
n = widgets.FloatSlider(value=n, min=0.9, max=2.0, step=0.1, description="n:", disabled=False))

In [13]:
display(widgets.HBox([fig1, wdj1]))